In [ ]:
import pandas as pd
import re

posts = pd.read_csv('Cleaned Posts/PostsOnly-wallstreetbets-NOAutoMod.csv')
comments = pd.read_csv('Cleaned Comments/CommentsOnly-wallstreetbets-NOAutoMod.csv')

postsData = posts.copy()
postsData['title'] = posts['title'].fillna('').astype(str) + " " + posts['selftext'].fillna('').astype(str)



# bodyComments = comments['body']
# postsData = postsData['title']

# print(bodyComments.shape)
# print(postsData.shape)

# subData = postsData.append(bodyComments)

# print(subData.shape)


documentArray = []


linkIDs = comments['link_id'].unique().tolist()

counter = 0

for item in linkIDs:
    returnString = ""
    
    
    ###Get the comments of the specific post link_id
    commentsofPost = comments[comments['link_id'] == item].fillna('').astype(str)
    for index, row in commentsofPost.iterrows():
        #Remove URLs from strings
        singleComment = str(row['body'])
        removedURL = re.sub(r"http\S+", "", singleComment)
        returnString += removedURL + " "
    ################################################
    
    
    ###Get Post and post body of post link_id 
    postTitleBody = postsData[postsData['id'] == item].fillna('').astype(str)
    postString = str(postTitleBody.iloc[0]['title']) + " " + str(postTitleBody.iloc[0]['selftext'])
    postString = re.sub(r"http\S+", "", postString)
    returnString = postString + returnString
    ################################################

    documentArray.append(str(returnString))

#Convert to pandas
subDocuments = pd.DataFrame(documentArray)
subDocuments = subDocuments.squeeze()
print(subDocuments.shape)

In [ ]:
#Data Pre-processing

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np


import nltk
nltk.download('wordnet')



stemmer = PorterStemmer()


my_stop_words = STOPWORDS.union(set(['like']))

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        # if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
        if token not in my_stop_words and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result


# doc_sample = selfTextPosts.iloc[5]

# print('original document: ')
# words = []
# for word in doc_sample.split(' '):
#     words.append(word)
# print(words)
# print('\n\n tokenized and lemmatized document: ')
# print(preprocess(doc_sample))



processed_docs = subDocuments.map(preprocess)
print(processed_docs[:10])

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(bow_corpus[41])


bow_doc_80 = bow_corpus[41]
for i in range(len(bow_doc_80)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_80[i][0], 
                                               dictionary[bow_doc_80[i][0]], bow_doc_80[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=8, id2word=dictionary, passes=20, iterations=400, workers=4)

# print(lda_model.print_topics(num_topics=5, num_words=5))

for idx, topic in lda_model.print_topics(num_topics=7, num_words=7):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=8, id2word=dictionary,passes=20, iterations=400, workers=4)

for idx, topic in lda_model_tfidf.print_topics(num_topics=7, num_words=7):
    print('Topic: {} Word: {}'.format(idx, topic))